In [2]:
import tensorflow as tf
import numpy as np


class TextCNN(object):
    '''
    A CNN for text classification
    Layers: 
        Embedding layer
        Convolutional layer
        Max-pooling layer
        Softmax layer
    '''
    def __init__(
        self, sequence_length, num_classes, vocab_size,
        embedding_size, filter_sizes, num_filters, l2_reg_lambda=0.0):

        
        # Placeholders
        self.input_x = tf.placeholder(tf.int32, [None, sequence_length], name="input_x")
        self.input_y = tf.placeholder(tf.float32, [None, num_classes], name="input_y")
        self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")
        
        # Keeping track of l2 regularization loss (optional)
        l2_loss = tf.constant(0.0)
        
        # Embedding Layer: Map vocabulary word indices with low-dimentional vector representations.
        # tf.device('/cpu:0'), 
        # we learn embedding matrix during training
        
        with tf.name_scope("embedding"):
            self.embedding_matrix = tf.Variable(
                tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0),
                name="Embedding_matrix")
            self.embedded_chars = tf.nn.embedding_lookup(self.embedding_matrix, self.input_x)
            
            # TensorFlow’s convolutional conv2d operation expects a 4-dimensional tensor with dimensions = batch, width, height and channel. 
            # The result of our embedding doesn’t contain the channel dimension, so we add it manually, 
            # leaving us with a layer of shape [None, sequence_length, embedding_size, 1].
            self.embedded_chars_expanded = tf.expand_dims(self.embedded_chars, -1)
           
        # Create a convolution + maxpool layer for each filter size
        
        
        # size is number of filters = len(filter_sizes)
        conv_output = []
        relu_output = []
        pooled_outputs = []
        
        for i, filter_size in enumerate(filter_sizes):
            with tf.name_scope("conv-maxpool-%s" % filter_size):
                
                # Convolution Layer
                filter_shape = [filter_size, embedding_size, 1, num_filters]
                filter_matrix = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="filter_matrix")
                b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
                
                conv = tf.nn.conv2d(
                    self.embedded_chars_expanded,
                    filter_matrix,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")
                
                conv_output.append(conv)
                
                # Apply nonlinearity
                h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
                
                relu_output.append(h)
                
                # Maxpooling over the outputs
                #pooled =  a feature vector, where the last dimension corresponds to our features.
                # tensor of shape [batch_size, 1, 1, num_filters]
                pooled = tf.nn.max_pool(
                    h,
                    ksize=[1, sequence_length - filter_size + 1, 1, 1],
                    strides=[1, 1, 1, 1],
                    padding='VALID',
                    name="pool")
                #pooled_outputs = [batch_size, num_filters_total]
                pooled_outputs.append(pooled)
        
        num_filters_total = num_filters * len(filter_sizes) 
        # Combine all the conv features
        self.conv_output_flat = tf.reshape(conv_output, [-1, num_filters_total])
        
        # Combine all the relu features
        self.relu_output_flat = tf.reshape(tf.concat(relu_output, 3), [-1, num_filters_total])
        

        # Combine all the pooled features
        self.h_pool = tf.concat(pooled_outputs, 3)
        self.h_pool_flat = tf.reshape(self.h_pool, [-1, num_filters_total])
        
        # Add dropout
        with tf.name_scope("dropout"):
            self.h_drop = tf.nn.dropout(self.h_pool_flat, self.dropout_keep_prob)
            
        # Scores
        with tf.name_scope("output"):
            W = tf.get_variable(
                "W",
                shape=[num_filters_total, num_classes],
                initializer=tf.contrib.layers.xavier_initializer())
            b = tf.Variable(tf.constant(0.1, shape=[num_classes]), name="b")
            l2_loss += tf.nn.l2_loss(W)
            l2_loss += tf.nn.l2_loss(b)
            self.scores = tf.nn.xw_plus_b(self.h_drop, W, b, name="scores")
            self.predictions = tf.argmax(self.scores, 1, name="predictions")

        
        # Loss and accuracy
        # Calculate mean cross-entropy loss
        with tf.name_scope("loss"):
            losses = tf.nn.softmax_cross_entropy_with_logits(logits=self.scores, labels=self.input_y)
            self.loss = tf.reduce_mean(losses) + l2_reg_lambda * l2_loss
        # Calculate accuracy
        with tf.name_scope("accuracy"):
            correct_predictions = tf.equal(self.predictions, tf.argmax(self.input_y, 1))
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")